In [8]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/home/hji20/.kaggle’: File exists


In [2]:
!kaggle competitions download -c open-problems-single-cell-perturbations

100%|█████████████████████████████████████▉| 3.82G/3.83G [00:53<00:00, 95.9MB/s]
100%|██████████████████████████████████████| 3.83G/3.83G [00:53<00:00, 76.3MB/s]


In [3]:
!mkdir train
!unzip open-problems-single-cell-perturbations.zip -d train

Archive:  open-problems-single-cell-perturbations.zip
  inflating: train/adata_excluded_ids.csv  
  inflating: train/adata_obs_meta.csv  
  inflating: train/adata_train.parquet  
  inflating: train/de_train.parquet  
  inflating: train/id_map.csv        
  inflating: train/multiome_obs_meta.csv  
  inflating: train/multiome_train.parquet  
  inflating: train/multiome_var_meta.csv  
  inflating: train/sample_submission.csv  


In [3]:
import pandas as pd

In [4]:
de_train = pd.read_parquet(f"./train/de_train.parquet")
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [6]:
from sklearn.preprocessing import OneHotEncoder
from rdkit import Chem
from rdkit.Chem import RDKFingerprint
import numpy as np

In [7]:
cell_types = de_train.iloc[:,0:1]

In [8]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(cell_types)
cell_types_ohe = ohe.transform(cell_types)
cell_types_ohe = cell_types_ohe.tolist()
de_train['cell_type_ohe'] = cell_types_ohe
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_ohe
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"


In [9]:
perts = de_train["SMILES"]
print(len(perts))
perts = perts.dropna()
print(len(perts))

614
614


In [10]:
cs_arr = []
for pert in perts:
  try:
    cs = Chem.CanonSmiles(pert)
    cs_arr.append(cs)
  except:
    print('Invalid SMILES: ', pert)
print(len(cs_arr))

614


In [11]:
mols = [Chem.MolFromSmiles(x) for x in cs_arr]
fps = [np.array(RDKFingerprint(mol)) for mol in mols]
de_train['fingerprint'] = fps
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_ohe,fingerprint
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."


In [12]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [14]:
res_arr = []
for i in range(len(de_train)):
  ct_ohe = np.array(de_train.iloc[i,-2:-1][0])
  fingerprint = np.array(de_train.iloc[i,-1:][0])
  res = np.concatenate((ct_ohe, fingerprint), axis=0)
  res_arr.append(res)

In [15]:
de_train['merged'] = res_arr

In [16]:
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_ohe,fingerprint,merged
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."


In [13]:
X = np.array(de_train.iloc[:,-1:], dtype=object)
Y = np.array(de_train.iloc[:,5:-3], dtype=object)

In [17]:
tmp_x = de_train.iloc[:,-1:]
#tmp_x = dup_final_df.iloc[:,-1:]
arr = np.empty(len(tmp_x), dtype=object)
for i in range(len(tmp_x)):
  arr[i] = tmp_x.iloc[i,0]
X = arr
print(len(X[1]))

2054


In [18]:
X = np.vstack(X)
X.shape

(614, 2054)

In [21]:
tmp_y = de_train.iloc[:,5:-3]
arr = np.empty(len(tmp_y), dtype=object)
for i in range(len(tmp_y)):
  arr[i] = tmp_y.iloc[i].values
Y = arr
print(len(Y[1]))

18211


In [22]:
tmp_y = de_train.iloc[:,5:105]
#tmp_y = dup_final_df.iloc[:,5:105]
#tmp_y = dup_final_df.iloc[:,5:-3]
arr = np.empty(len(tmp_y), dtype=object)
for i in range(len(tmp_y)):
  arr[i] = tmp_y.iloc[i].values
Y = arr
print(len(Y[1]))

18211


In [23]:
Y = np.vstack(Y)
Y.shape

(886, 18211)

In [63]:
models = []

In [15]:
de_train["cell_type"].value_counts()

cell_type
NK cells              146
T cells CD4+          146
T regulatory cells    146
T cells CD8+          142
B cells                17
Myeloid cells          17
Name: count, dtype: int64

In [16]:
tmp = de_train[de_train["cell_type"] == "B cells"]
duplicated_df = pd.concat([tmp] * 8, ignore_index=True)
len(duplicated_df)

136

In [17]:
tmp2 = de_train[de_train["cell_type"] == "Myeloid cells"]
duplicated_df2 = pd.concat([tmp2] * 8, ignore_index=True)
len(duplicated_df2)

136

In [18]:
dup_final_df = pd.concat([de_train, duplicated_df, duplicated_df2], ignore_index=True)
dup_final_df["cell_type"].value_counts()

cell_type
B cells               153
Myeloid cells         153
NK cells              146
T cells CD4+          146
T regulatory cells    146
T cells CD8+          142
Name: count, dtype: int64

In [24]:
models_2 = []
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_split, y_train_split, test_size=0.25, random_state=42)

# Train multiple LightGBM models (one for each output element)
for i in tqdm(range(Y.shape[1])):

    train_data = lgb.Dataset(X_train, label=y_train[:, i], )  # Train a model for each output element
    val_data = lgb.Dataset(X_val, label=y_val[:, i], reference=train_data) 
    params = {
        'objective': 'regression',
        'metric': 'l2',
        'num_leaves': 100,
        'max_depth':-1,
        'learning_rate': 0.1,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'num_threads': 16,
        'early_stopping_rounds': 10,
        'boosting_type': 'gbdt',
    }
    num_round = 1000
    model = lgb.train(params, train_data, num_round, valid_sets=[val_data])
    models_2.append(model)

# Make predictions on the test set
y_pred = np.array([model.predict(X_test_split) for model in models_2]).T  # Transpose predictions

# Display the predicted values
print("Predicted values:")
print(y_pred)

100%|██████████| 18211/18211 [2:44:40<00:00,  1.84it/s]  


Predicted values:
[[ 6.33080789e-02  4.76195864e-01  1.77786045e-01 ...  2.21695932e-01
  -1.07370507e-01  2.97633029e-01]
 [-1.62491610e-02 -1.44790298e-01  6.01863207e-01 ...  2.31448177e-01
   1.08956346e-01  9.37711608e-02]
 [ 5.01986028e+00  1.29616500e+00  7.12971396e+00 ...  2.77354983e-01
   2.42248278e-01 -1.80483788e-01]
 ...
 [ 5.97825766e-01 -5.18904993e-03  9.51961341e-01 ...  4.93829878e-01
  -2.88585762e-02 -1.38847603e-01]
 [ 1.84905467e-01  1.18972818e-01  6.40690782e-01 ...  2.84949680e-01
   2.19230322e-01  1.89936053e-01]
 [ 3.75781842e-01 -1.40786345e-02  4.97845542e-01 ...  8.58085826e-01
  -4.45776550e-02 -1.62301371e-01]]


In [31]:
for i in range(len(models_2)):
  m = models_2[i]
  fn = "trained_models_2/trained_model_" + str(i) + ".txt"
  m.save_model(fn)

In [32]:
mse = mean_squared_error(y_pred.flatten(), y_test_split.flatten())  # Flatten arrays to 1D
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 4.4433615128344925


In [33]:
mse / len(y_test_split)

0.024962705128283667

In [102]:
mse = mean_squared_error(y_pred.flatten(), y_test_split.flatten())  # Flatten arrays to 1D
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.8671805175799607


In [164]:
mse = mean_squared_error(y_pred.flatten(), y_test_split.flatten())  # Flatten arrays to 1D
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 3.990610630607609


In [222]:
mse = mean_squared_error(y_pred.flatten(), y_test_split.flatten())  # Flatten arrays to 1D
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 3.976134215039349


In [225]:
!mkdir trained_models_2

In [79]:
for i in range(len(models)):
  m = models[i]
  fn = "trained_models/trained_model_" + str(i) + ".txt"
  m.save_model(fn)

Without oversampling:

In [80]:
mse = mean_squared_error(y_pred.flatten(), y_test_split.flatten())  # Flatten arrays to 1D
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 2.8654083933524506


In [81]:
mse / len(y_test_split)

0.023296003197987403

In [34]:
id_map = pd.read_csv("./train/id_map.csv", sep=",", dtype=str)
id_map.columns
id_map.head()

,id,cell_type,sm_name
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...
1,1,B cells,ABT-199 (GDC-0199)
2,2,B cells,ABT737
3,3,B cells,AMD-070 (hydrochloride)
4,4,B cells,AT 7867


In [35]:
ohe_col = []
for x in id_map.cell_type:
  if x == "B cells":
    ohe_col.append([1.0, 0.0, 0.0, 0.0, 0.0, 0.0])
  else:
    ohe_col.append([0.0, 1.0, 0.0, 0.0, 0.0, 0.0])
id_map["ohe_col"] = ohe_col
id_map.head()

,id,cell_type,sm_name,ohe_col
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,1,B cells,ABT-199 (GDC-0199),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,2,B cells,ABT737,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,3,B cells,AMD-070 (hydrochloride),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,4,B cells,AT 7867,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [40]:
cp_df = de_train[["sm_name", "SMILES"]].drop_duplicates()
cp_df.head()

,sm_name,SMILES
0,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1
4,Mometasone Furoate,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...
8,Idelalisib,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...
14,Vandetanib,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1
18,Bosutinib,COc1cc(Nc2c(C#N)cnc3cc(OCCCN4CCN(C)CC4)c(OC)cc...


In [42]:
cs_arr = []
sm_arr = cp_df["SMILES"]
for sm in sm_arr:
  try:
    cs = Chem.CanonSmiles(sm)
    cs_arr.append(cs)
  except:
    print('Invalid SMILES: ', sm)
print(len(cs_arr))

146


In [43]:
mols = [Chem.MolFromSmiles(x) for x in cs_arr]
fps = [np.array(RDKFingerprint(mol)) for mol in mols]
cp_df['fingerprint'] = fps
cp_df.head()

,sm_name,SMILES,fingerprint
0,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,"[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,Mometasone Furoate,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,"[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."
8,Idelalisib,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, ..."
14,Vandetanib,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,"[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, ..."
18,Bosutinib,COc1cc(Nc2c(C#N)cnc3cc(OCCCN4CCN(C)CC4)c(OC)cc...,"[1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, ..."


In [44]:
cp_df["fingerprint"] = cp_df["fingerprint"].apply(lambda x: np.array(x))
cp_df.head()

,sm_name,SMILES,fingerprint
0,Clotrimazole,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,"[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,Mometasone Furoate,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,"[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."
8,Idelalisib,CC[C@H](Nc1ncnc2[nH]cnc12)c1nc2cccc(F)c2c(=O)n...,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, ..."
14,Vandetanib,COc1cc2c(Nc3ccc(Br)cc3F)ncnc2cc1OCC1CCN(C)CC1,"[1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, ..."
18,Bosutinib,COc1cc(Nc2c(C#N)cnc3cc(OCCCN4CCN(C)CC4)c(OC)cc...,"[1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, ..."


In [45]:
id_map_merged = pd.merge(id_map, cp_df, on="sm_name", how='inner')
id_map_merged.head()

,id,cell_type,sm_name,ohe_col,SMILES,fingerprint
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ..."
1,128,Myeloid cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ..."
2,1,B cells,ABT-199 (GDC-0199),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
3,129,Myeloid cells,ABT-199 (GDC-0199),"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ..."
4,2,B cells,ABT737,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...,"[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."


In [46]:
len(id_map)

255

In [47]:
len(id_map_merged)

255

In [49]:
res_arr = []
for i in range(len(id_map_merged)):
  ct_ohe = np.array(id_map_merged.iloc[i,-3:-2][0])
  fingerprint = np.array(id_map_merged.iloc[i,-1:][0])
  res = np.concatenate((ct_ohe, fingerprint), axis=0)
  res_arr.append(res)

In [50]:
id_map_merged['merged'] = res_arr
id_map_merged.head()

,id,cell_type,sm_name,ohe_col,SMILES,fingerprint,merged
0,0,B cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
1,128,Myeloid cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
2,1,B cells,ABT-199 (GDC-0199),"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, ..."
3,129,Myeloid cells,ABT-199 (GDC-0199),"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,"[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, ..."
4,2,B cells,ABT737,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",CN(C)CC[C@H](CSc1ccccc1)Nc1ccc(S(=O)(=O)NC(=O)...,"[0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."


In [51]:
tmp_id_x = id_map_merged.iloc[:,-1:]
arr = np.empty(len(tmp_id_x), dtype=object)
for i in range(len(tmp_id_x)):
  arr[i] = tmp_id_x.iloc[i,0]
id_X = arr
print(len(id_X[1]))

2054


In [52]:
id_X = np.vstack(id_X)
id_X.shape

(255, 2054)

In [53]:
genes = np.array(de_train.columns[5:18216])
genes[-1]

'ZZEF1'

In [54]:
id_y_pred = np.array([model.predict(id_X) for model in models_2]).T
id_y_pred.shape

(255, 18211)

In [84]:
id_y_pred = np.array([model.predict(id_X) for model in models]).T

In [55]:
id_y_pred.shape

(255, 18211)

In [56]:
out_fh = open("./submission2.csv", 'w')
out_fh.write("id")
for g in genes:
  out_fh.write("," + g)
out_fh.write("\n")

1

In [57]:
ctr = 0
for row in id_y_pred:
    out_fh.write(str(ctr))
    for col in row:
        out_fh.write(',' + str(col))
    out_fh.write('\n')
    ctr += 1

In [58]:
out_fh.close()

In [1]:
!kaggle competitions submit -c open-problems-single-cell-perturbations -f ./submission2.csv -m "lgbm"

/bin/bash: kaggle: command not found
